Nama : Rival Moh. Wahyudi
Email : vallllwhy@students.unnes.ac.id
Domisili Kota : Semarang

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import re
import string

#install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #penghilangan imbuhan kata dalam bahasa indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory 

from fuzzywuzzy import fuzz, process

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, BatchNormalization, GRU, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import os
import datetime

#Data Preparation

In [4]:
df = pd.read_csv('jobstreet_review.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20001 entries, 0 to 64132
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  20001 non-null  object
dtypes: object(1)
memory usage: 312.5+ KB


In [5]:
#mencari data duplikat
df.duplicated().sum()

1053

In [6]:
df.isna().sum()

content    0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20001 entries, 0 to 64132
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  20001 non-null  object
dtypes: object(1)
memory usage: 312.5+ KB


#Prepocessing Texts

In [8]:
#fungsi Tokenizing
def tokenizingText(text):
    text = text.split()
    return text

In [5]:
#fungsi menggabungkan teks yang tadinya menjadi list-list
def toSentence(list_word):
    sentence = ' '.join(word for word in list_word)
    return sentence

In [10]:
#fungsi cleaning text -->   menghapus mention, hashtag, retweet, tautan, angka dan tanda baca, selai itu new line akan diganti dengan spasi, spasi extra di awal dan akhir akan dihapus

def claningtext(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) #menghapus mention dan menjadikan semua huruf menjadi huruf kecil
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus Hashtag
    # menghilangkan underscore pada teks yang bergabung menjadi satu dengan underscore
    text = re.sub(r'::', ' ', text)
    text = re.sub(r'_', ' ', text)
    text = re.sub(r'RT[\s]', '', text) # menghapus retweet
    text = re.sub(r'http\S+', '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghpaus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.replace('  ', ' ') #menghapus double space menjadi satu
    text = text.translate(str.maketrans('', '', string.punctuation)) # Menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    
    return text

In [11]:
# penerjemahan emoji dan emoticon
import emoji

def emojisEmoticon(text):
    Emoticonsdict = {
        ":)": "senang",
        ":-)": "senang",
        ":D": "senang",
        ":-D": "senang",
        ":(": "sedih",
        ":-(": "sedih",
        ":/": "kecewa",
        ":-/": "kecewa",
        ";)": "kedip",
        ";-)": "kedip"
    }

    for emoticon, meaning in Emoticonsdict.items():
        text = text.lower().replace(emoticon, meaning)

    return emoji.demojize(text, language='id')

In [12]:
# menghilangkan stopword
def wordStop(text):
    word_stop = pd.read_csv('combined_stop_words.txt')
    word_stop = word_stop['stop_words'].tolist()
    listStopword_id = set(stopwords.words('indonesian'))
    listStopword_en = set(stopwords.words('english'))
    listStopword_id.update(listStopword_en)
    listStopword_id.update(word_stop)
    listStopword_id.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for word in text:
        if word not in listStopword_id:
            filtered.append(word)
    
    return filtered

In [13]:
# melakukan stemming word
def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    stemmed_words = [stemmer.stem(word) for word in text]

    return stemmed_words

In [14]:
#menggnati slangword dengan kata yang sebenarnya

def replaceSlang(text):
    slang_text = {"@": "di", "abis": "habis", "ad": "ada", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}

    text = [slang_text.get(word, word) for word in text]

        
    return text

In [15]:
#mengaplikasikan fuzzy algoritme agar kata kata dengan pelafalan salah dapat diganti dengan yang lebih benar

def fuzzyText(text):
    def fuzz(word, root):
        best_match = process.extractOne(word, root)
        return best_match
    root_text = pd.read_csv('combined_root_words.txt')
    root_text = root_text['root_word'].tolist()
    english_root = set(stopwords.words('english'))
    root_text.append(english_root)

    text = [fuzz(words, root_text) for words in text]

    return text

saatnya membersihkan data dengan fungsi yang telah di buat sebelumnya

In [16]:
df['hapus_emot'] = df['content'].apply(emojisEmoticon)

In [17]:
df['cleaning'] = df['hapus_emot'].apply(claningtext)

In [18]:
df['token'] = df['cleaning'].apply(tokenizingText)

In [19]:
df['slang'] = df['token'].apply(replaceSlang)

In [20]:
df['stopword'] = df['slang'].apply(wordStop)

In [21]:
df['stemming'] = df['stopword'].apply(stemmingText)

In [70]:
df = pd.read_csv('checkpoint.csv', converters={
    'stemming':pd.eval,
    'token' : pd.eval,
    'slang' : pd.eval,
    'stopword' : pd.eval
})

In [71]:
df['text_akhir'] = df['stemming'].apply(toSentence)

In [72]:
df.to_csv('checkpoint.csv', index=False)

#Proses Pelabelan kata scoring sentiment

scoring dilakukan dengan 3 kategori, yaitu Netral, Positif, dan Negatif

metode nantinya dilakukan dengan one-hot encoding

In [33]:
df.head()

,content,hapus_emot,cleaning,token,slang,stopword,stemming,text_akhir
0,"Aplikasinya bagus & lengkap, pernah dapat kerj...","aplikasinya bagus & lengkap, pernah dapat kerj...",aplikasinya bagus lengkap pernah dapat kerjaan...,"[aplikasinya, bagus, lengkap, pernah, dapat, k...","[aplikasinya, bagus, lengkap, pernah, dapat, k...","[aplikasinya, bagus, lengkap, kerjaan, dr, suk...","[aplikasi, bagus, lengkap, kerja, dr, suka, ta...",aplikasi bagus lengkap kerja dr suka tampil ba...
1,"Aku ga tahu ya, selalu dihubungi perusahaan bo...","aku ga tahu ya, selalu dihubungi perusahaan bo...",aku ga tahu ya selalu dihubungi perusahaan bod...,"[aku, ga, tahu, ya, selalu, dihubungi, perusah...","[aku, ga, tahu, iya, selalu, dihubungi, perusa...","[dihubungi, perusahaan, bodong, apply, perusah...","[hubung, usaha, bodong, apply, usaha, tsb, kal...",hubung usaha bodong apply usaha tsb kalimat ca...
2,Saya kecewa dengan apk jobstreet sekarang tida...,saya kecewa dengan apk jobstreet sekarang tida...,saya kecewa dengan apk jobstreet sekarang tida...,"[saya, kecewa, dengan, apk, jobstreet, sekaran...","[saya, kecewa, dengan, apk, jobstreet, sekaran...","[kecewa, apk, jobstreet, terpercaya, apply, pa...","[kecewa, apk, jobstreet, percaya, apply, part,...",kecewa apk jobstreet percaya apply part time b...
3,"Saya lebih suka jobstreet versi sebelumnya, di...","saya lebih suka jobstreet versi sebelumnya, di...",saya lebih suka jobstreet versi sebelumnya dim...,"[saya, lebih, suka, jobstreet, versi, sebelumn...","[saya, lebih, suka, jobstreet, versi, sebelumn...","[suka, jobstreet, versi, masukkan, lamaran, pe...","[suka, jobstreet, versi, masuk, lamar, usaha, ...",suka jobstreet versi masuk lamar usaha si rekr...
4,"Belum adanya tipe magang, dalam judul sudah di...","belum adanya tipe magang, dalam judul sudah di...",belum adanya tipe magang dalam judul sudah dic...,"[belum, adanya, tipe, magang, dalam, judul, su...","[belum, adanya, tipe, magang, dalam, judul, su...","[tipe, magang, judul, dicari, maganginternship...","[tipe, magang, judul, cari, maganginternship, ...",tipe magang judul cari maganginternship kerja ...


In [73]:
def txt_to_dict(file_path):
    result_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) >= 2:
                key = parts[0]
                value = ' '.join(parts[1:])  # in case value has spaces
                result_dict[key] = int(value)
    return result_dict

In [74]:
lexicon = txt_to_dict('sentimentword.txt')

In [75]:

def sentimentWord(text):
    score = 0
    text = text.split()
    for words in text:
        if words in lexicon:
            score = score + lexicon[words]

    return score

In [76]:
def analysis_word(scoreText):
    if (scoreText == 0):
        return 'Netral'
    elif (scoreText > 0):
        return 'Positif'
    else:
        return 'Negatif'

In [77]:
#apply score ke dataframe
df['score_text'] = df['text_akhir'].apply(sentimentWord)

In [78]:
df['sentiment'] = df['score_text'].apply(analysis_word)

In [79]:
df.to_csv('final.csv', index=False)

In [80]:
df =pd.read_csv('final.csv')

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   content     20000 non-null  object
 1   hapus_emot  20000 non-null  object
 2   cleaning    20000 non-null  object
 3   token       20000 non-null  object
 4   slang       20000 non-null  object
 5   stopword    20000 non-null  object
 6   stemming    20000 non-null  object
 7   text_akhir  19958 non-null  object
 8   score_text  20000 non-null  int64 
 9   sentiment   20000 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.5+ MB


#membuat data latih dan data test

In [3]:
#sebagai akses data kembali untuk data latih dan data set
df = pd.read_csv('final.csv', converters={
    'stemming':pd.eval,
    'token' : pd.eval,
    'slang' : pd.eval,
    'stopword' : pd.eval,
    'score_text' : pd.eval
})

In [6]:
df['text_akhir'] = df['stemming'].apply(toSentence)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   content     20000 non-null  object
 1   hapus_emot  20000 non-null  object
 2   cleaning    20000 non-null  object
 3   token       20000 non-null  object
 4   slang       20000 non-null  object
 5   stopword    20000 non-null  object
 6   stemming    20000 non-null  object
 7   text_akhir  20000 non-null  object
 8   score_text  20000 non-null  int64 
 9   sentiment   20000 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.5+ MB


In [8]:
df['sentiment'].value_counts()

sentiment
Netral     9598
Positif    7990
Negatif    2412
Name: count, dtype: int64

In [8]:
#one hot encoding untuk kolom sentiment

category = pd.get_dummies(df.sentiment, dtype=int)
category

,Negatif,Netral,Positif
0,0,0,1
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
...,...,...,...
19995,0,0,1
19996,0,1,0
19997,0,0,1
19998,0,0,1


In [9]:
example_text = df['text_akhir'].tolist()
print (example_text)

['aplikasi bagus lengkap kerja dr suka tampil baharu cepat ngintip lamar aplikasi profil usaha lowong kerja usaha aplikasi baru error koneksi internet bagus semenjak masuk lamar jobstreet telpon wa masuk dr tipu', 'hubung usaha bodong apply usaha tsb kalimat cantum data app amanin udh aktif mode comot data bodong apply mohon tingkat aman terimakasih', 'kecewa apk jobstreet percaya apply part time bioskop cgv poins wa tiktok affiliate kerja online ngelike postingan balas job apply contoh pokok apply wa guna balas', 'suka jobstreet versi masuk lamar usaha si rekruter tarik profil tera terang profil x x dan terus tanda hubung si rekruter si rektuter cocok profil versi cuman terang terang lihat xxx kurang versi sekrang versi fitur dimasukin', 'tipe magang judul cari maganginternship kerja aplikasi magang maximal mohon tambah magang kerja', 'bagus aplikasi sngt sayang formalitas usaha menginklankan jobstreet lamar jobstreet usaha apply lihat respon lamar batas buka lamar tutup', 'jobstreet 

In [10]:
#sebelum benar-benar dipakai kita ekstraksi fitur terlebih dahulu data categoricalnya dengna tf-idf

tfidf = TfidfVectorizer(max_features=1100)
#keduanaya sama saja data dan label
X_tfidf = tfidf.fit_transform(example_text).toarray()
y = np.array(category)

In [11]:
#konversi hasil ekstraksi fitur menjadi dataframe

features_df = pd.DataFrame(X_tfidf, columns = tfidf.get_feature_names_out()) # column untuk memberikan nama column sesua dengan kumpulan kata yang sudah di diapat pada tahap ekstraksi fitur
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 1100 entries, aamiin to zaman
dtypes: float64(1100)
memory usage: 167.8 MB


In [12]:
X_tfidf = np.expand_dims(X_tfidf, axis=1)

In [13]:
#membagi data menjadi data latih dan data test harus sudah dalam bentuk array
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = .2, random_state = 42)

In [14]:
print (X_train.shape)

(16000, 1, 1100)


In [15]:
print (X_test.shape)

(4000, 1, 1100)


#Build The Model

penambahan callback(monitoring untuk pelatihan) dan batch normalization(pengaturan batch pada latihan), simpleRNN, dropout(menonaktifkan random neuron pada layer)

Memakai model RNN

In [16]:
model = Sequential([
    Input(shape=(1, 1100)),
    SimpleRNN(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense (64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense (3, activation='softmax')
])

In [17]:
#compile model

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
his = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=.2)

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5670 - loss: 1.0062 - val_accuracy: 0.8050 - val_loss: 0.5446
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8490 - loss: 0.3994 - val_accuracy: 0.9231 - val_loss: 0.2148
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9093 - loss: 0.2468 - val_accuracy: 0.9378 - val_loss: 0.1739
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9374 - loss: 0.1867 - val_accuracy: 0.9422 - val_loss: 0.1602
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9438 - loss: 0.1733 - val_accuracy: 0.9459 - val_loss: 0.1576
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9545 - loss: 0.1388 - val_accuracy: 0.9453 - val_loss: 0.1589
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9612 - loss: 0.1207 - val_accuracy: 0.9491 - val_loss: 0.1656
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9636 - loss: 0.1137 - val_accuracy: 0.

In [19]:
model.evaluate(X_test, y_test, batch_size=20)

200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9545 - loss: 0.2353


[0.22681859135627747, 0.9537500143051147]

Algoritma LSTM

In [20]:
model1 = Sequential([
    Input(shape=(1, 1100)),
    LSTM(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense (3, activation='softmax')
])

In [21]:
model1.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
his1 = model1.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=.2)

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6303 - loss: 0.8812 - val_accuracy: 0.8425 - val_loss: 0.5366
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9086 - loss: 0.2545 - val_accuracy: 0.9337 - val_loss: 0.1822
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9468 - loss: 0.1562 - val_accuracy: 0.9447 - val_loss: 0.1605
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9590 - loss: 0.1202 - val_accuracy: 0.9484 - val_loss: 0.1615
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9696 - loss: 0.0903 - val_accuracy: 0.9500 - val_loss: 0.1656
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9756 - loss: 0.0736 - val_accuracy: 0.9509 - val_loss: 0.1662
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9762 - loss: 0.0685 - val_accuracy: 0.9522 - val_loss: 0.1753
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9834 - loss: 0.0550 - val_accuracy: 

In [23]:
model1.evaluate(X_test, y_test)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9527 - loss: 0.2753


[0.2516437768936157, 0.9539999961853027]

Algoritma GRU

In [24]:
model2 = Sequential([
    Input(shape=(1, 1100)),
    GRU(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense (3, activation='softmax')
])

In [25]:
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
his2 = model2.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=.2)

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.6297 - loss: 0.9154 - val_accuracy: 0.8269 - val_loss: 0.5414
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9087 - loss: 0.2638 - val_accuracy: 0.9316 - val_loss: 0.1903
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9551 - loss: 0.1474 - val_accuracy: 0.9416 - val_loss: 0.1773
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9639 - loss: 0.1060 - val_accuracy: 0.9466 - val_loss: 0.1687
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9713 - loss: 0.0915 - val_accuracy: 0.9472 - val_loss: 0.1838
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9750 - loss: 0.0817 - val_accuracy: 0.9453 - val_loss: 0.1847
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9784 - loss: 0.0631 - val_accuracy: 0.9494 - val_loss: 0.1768
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9804 - loss: 0.0579 - val_accuracy: 0

In [27]:
model2.evaluate(X_test, y_test)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9500 - loss: 0.3050


[0.28142011165618896, 0.9492499828338623]

Kesimpulan dari ketiga model yang menggunakan metode ekstraksi fitur yang sama yaitu TF-IDF dan menggunakan beberapa algoritma deeplerning yang didalamanya juga dibedakan dengan diberikan algoritma RNN, LSTM, dan GRU untuk ketiga model secara terpisah didapatkan bahwa prose fitting yang paling cepat adalah saat model menggunakan algoritma SimpleRNN dengan rata-rata waktu 2 detik untuk sekali epochs, dan dari ketiga model yang dibuat mencapai tingkat akurasi kurang lebih sama, yaitu 95%